In [1]:
import pickle
import numpy as np
from scipy import stats
import pandas as pd
import numpy as np
from time import time
from IPython.display import Image
from tqdm.notebook import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
final_dataset = pd.read_csv('/content/drive/My Drive/PdM data/final_dataset.csv')

In [4]:
final_dataset.shape

(291341, 34)

In [5]:
final_dataset.head()

,Unnamed: 0,machineID,datetime,volt_3H_mean,rotate_3H_mean,pressure_3H_mean,vibration_3H_mean,volt_3H_std,rotate_3H_std,pressure_3H_std,vibration_3H_std,volt_24H_mean,rotate_24H_mean,pressure_24H_mean,vibration_24H_mean,volt_24H_std,rotate_24H_std,pressure_24H_std,vibration_24H_std,error1count,error2count,error3count,error4count,error5count,comp1,comp2,comp3,comp4,age,failure,model1,model2,model3,model4
0,0,1,2015-01-02 06:00:00,180.133784,440.608320,94.137969,41.551544,21.322735,48.770512,2.135684,10.037208,169.733809,445.179865,96.797113,40.385160,11.233120,48.717395,10.079880,5.853209,0.0,0.0,0.0,0.0,0.0,20.000,215.000,155.000,170.000,18,none,0,0,1,0
1,1,1,2015-01-02 09:00:00,176.364293,439.349655,101.553209,36.105580,18.952210,51.329636,13.789279,6.737739,170.614862,446.364859,96.849785,39.736826,12.519402,48.385076,10.171540,6.163231,0.0,0.0,0.0,0.0,0.0,20.125,215.125,155.125,170.125,18,none,0,0,1,0
2,2,1,2015-01-02 12:00:00,160.384568,424.385316,99.598722,36.094637,13.047080,13.702496,9.988609,1.639962,169.893965,447.009407,97.715600,39.498374,13.370357,42.432317,9.471669,6.195076,0.0,0.0,0.0,0.0,0.0,20.250,215.250,155.250,170.250,18,none,0,0,1,0
3,3,1,2015-01-02 15:00:00,170.472461,442.933997,102.380586,40.483002,16.642354,56.290447,3.305739,8.854145,171.243444,444.233563,96.666060,40.229370,13.299281,41.346121,8.731229,5.687944,0.0,0.0,0.0,0.0,0.0,20.375,215.375,155.375,170.375,18,none,0,0,1,0
4,4,1,2015-01-02 18:00:00,163.263806,468.937558,102.726648,40.921802,17.424688,38.680380,9.105775,3.060781,170.792486,448.440437,95.766838,40.055214,13.954518,43.490234,8.061653,5.898069,0.0,0.0,0.0,0.0,0.0,20.500,215.500,155.500,170.500,18,none,0,0,1,0


In [6]:
final_dataset = final_dataset.drop('Unnamed: 0', axis = 1)

In [7]:
final_dataset['datetime'] = pd.to_datetime(final_dataset['datetime'], format='%Y-%m-%dT%H:%M:%S')
print (final_dataset)

        machineID            datetime  volt_3H_mean  ...  model2  model3  model4
0               1 2015-01-02 06:00:00    180.133784  ...       0       1       0
1               1 2015-01-02 09:00:00    176.364293  ...       0       1       0
2               1 2015-01-02 12:00:00    160.384568  ...       0       1       0
3               1 2015-01-02 15:00:00    170.472461  ...       0       1       0
4               1 2015-01-02 18:00:00    163.263806  ...       0       1       0
...           ...                 ...           ...  ...     ...     ...     ...
291336        100 2015-12-31 18:00:00    177.758754  ...       0       0       1
291337        100 2015-12-31 21:00:00    166.078945  ...       0       0       1
291338        100 2016-01-01 00:00:00    158.327984  ...       0       0       1
291339        100 2016-01-01 03:00:00    170.306956  ...       0       0       1
291340        100 2016-01-01 06:00:00    182.525427  ...       0       0       1

[291341 rows x 33 columns]


In [8]:
# https://stackoverflow.com/questions/54312802/pandas-convert-from-datetime-to-integer-timestamp
final_dataset.datetime = final_dataset.datetime.astype(int) / 10 ** 9 # convert from datetime to integer timestamp

In [9]:
import pickle

final = final_dataset.drop(['datetime', 'machineID', 'failure'], axis=1).columns.values

filename = '/content/drive/My Drive/PdM data/columns.sav'
pickle.dump(final, open(filename, 'wb'))

In [10]:
# Establish the datetime corresponding to the records that will be used for training and testing.
datetime = [[pd.to_datetime('2015-07-31 01:00:00'), pd.to_datetime('2015-08-01 01:00:00')],
                   [pd.to_datetime('2015-08-31 01:00:00'), pd.to_datetime('2015-09-01 01:00:00')],
                   [pd.to_datetime('2015-09-30 01:00:00'), pd.to_datetime('2015-10-01 01:00:00')],
                   [pd.to_datetime('2015-10-30 01:00:00'), pd.to_datetime('2015-11-01 01:00:00')]]

In [11]:
X = final_dataset.drop(['datetime', 'machineID', 'failure'], axis=1)
y = final_dataset.failure

In [12]:
from sklearn.model_selection import train_test_split, TimeSeriesSplit

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, stratify =final_dataset['failure'])
X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train)

In [13]:
X_train = final_dataset.drop(['datetime', 'machineID', 'failure'], axis =1 ).values

y_train = final_dataset.failure.values

## **Importing "final_finc_1"**

In [16]:
def final_func_1(final_dataset):
  """
  Input :: dataframe which consist of raw data

  Output :: Predicted values
  """
  start = time()
       
  ## importing files
  column_filename = '/content/drive/My Drive/PdM data/columns.sav'

  modelfilename = '/content/drive/My Drive/PdM data/best_model.sav'
  ## loading files
  final = pickle.load(open(column_filename, 'rb'))
  best_model = pickle.load(open(modelfilename,'rb'))
      

  ##  if input is np.ndarray or list or list of list
  if isinstance(final_dataset,(list,np.ndarray)):
    X_data = pd.DataFrame(data=final_dataset, columns=final)
    
  print('Time for Preprocessing and Featursiation is :: {}  sec' .format(time() - start))
    
  now = time()
        
  y_pred = best_model.predict(X_data)
    
  print('Time for inference is :: {}  sec' .format(time() - now))
    
  return y_pred

### **Array of single datapoint**

In [17]:
X_train[0]

array([180.13378397, 440.60832011,  94.1379695 ,  41.55154363,
        21.32273479,  48.77051197,   2.13568373,  10.03720806,
       169.7338089 , 445.1798646 ,  96.79711296,  40.38515999,
        11.23312028,  48.71739478,  10.07988023,   5.85320856,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,  20.        , 215.        , 155.        ,
       170.        ,  18.        ,   0.        ,   0.        ,
         1.        ,   0.        ])

In [19]:
final_func_1(X_train[0].reshape(1, - 1))

Time for Preprocessing and Featursiation is :: 0.004076719284057617  sec
Time for inference is :: 0.0019092559814453125  sec


array(['none'], dtype=object)

In [20]:
final_func_1(X_train)

Time for Preprocessing and Featursiation is :: 0.004692792892456055  sec
Time for inference is :: 0.04314613342285156  sec


array(['none', 'none', 'none', ..., 'none', 'none', 'none'], dtype=object)

## **Importing final_func_2(X,Y)**

In [21]:
def final_func_2(X, Y):
    
    """
    Input :: X (Features), Y (Target Values)
    
    Output :: accuracy_score
    
    """
    
    return accuracy_score(Y,final_func_1(X))

In [22]:
y_pred = final_func_1(X_train)

Time for Preprocessing and Featursiation is :: 0.005970001220703125  sec
Time for inference is :: 0.03919839859008789  sec


In [23]:
from sklearn.metrics import accuracy_score

accuracy_score_  = final_func_2(X_train, y_train)

Time for Preprocessing and Featursiation is :: 0.009193897247314453  sec
Time for inference is :: 0.045317888259887695  sec


In [24]:
print("The accuracy_score is :: ",accuracy_score_)

The accuracy_score is ::  0.998620173610992


In [25]:
y_pred_test = final_func_1(X_test.values)

Time for Preprocessing and Featursiation is :: 0.006985902786254883  sec
Time for inference is :: 0.016889572143554688  sec
